In [1]:
import pandas as pd
import os
from datetime import datetime

# Paths
data_path = '/content/data.csv'
transform_dir = '/content/transformed'
os.makedirs(transform_dir, exist_ok=True)

# Load Data
df = pd.read_csv(data_path)

# Preview data and confirm available columns
df.head(), df.columns


(         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
 0    842302         M        17.99         10.38          122.80     1001.0   
 1    842517         M        20.57         17.77          132.90     1326.0   
 2  84300903         M        19.69         21.25          130.00     1203.0   
 3  84348301         M        11.42         20.38           77.58      386.1   
 4  84358402         M        20.29         14.34          135.10     1297.0   
 
    smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
 0          0.11840           0.27760          0.3001              0.14710   
 1          0.08474           0.07864          0.0869              0.07017   
 2          0.10960           0.15990          0.1974              0.12790   
 3          0.14250           0.28390          0.2414              0.10520   
 4          0.10030           0.13280          0.1980              0.10430   
 
    ...  texture_worst  perimeter_worst  area_wo

In [6]:
# 1. Remove duplicates
df_full = df.drop_duplicates()

# 2. Handle missing values (fill numerics with mean)
df_full.fillna(df_full.mean(numeric_only=True), inplace=True)

# 3. Replace 'M' and 'B' with 'Malignant' and 'Benign' in 'diagnosis' (if exists)
if 'diagnosis' in df_full.columns:
    df_full['diagnosis'] = df_full['diagnosis'].replace({'M': 'Malignant', 'B': 'Benign'})

# 4. Add a new calculated column if columns exist
if 'mean_radius' in df_full.columns and 'mean_texture' in df_full.columns:
    df_full['radius_texture_product'] = df_full['mean_radius'] * df_full['mean_texture']

# Save transformed full data
full_save_path = os.path.join(transform_dir, 'transformed_full.csv')
df_full.to_csv(full_save_path, index=False)

print("Full transformation complete.")

# If id column exists for incremental check
if 'id' in df_full.columns:
    df_full['id'] = pd.to_datetime(df_full['id'])

    # Simulate last extraction time (for example purposes)
    last_extraction_time = pd.to_datetime('2025-01-01 00:00:00')

    # Filter records after the last extraction
    df_incremental = df_full[df_full['id'] > last_extraction_time]

    # Save transformed incremental data
    inc_save_path = os.path.join(transform_dir, 'transformed_incremental.csv')
    df_incremental.to_csv(inc_save_path, index=False)

    print("Incremental transformation complete.")

else:
    print("No id column found for incremental transformation.")



Full transformation complete.
Incremental transformation complete.


In [8]:
import pandas as pd
import sqlite3
import os

# Paths
full_data_path = '/content/transformed/transformed_full.csv'
incremental_data_path = '/content/transformed/transformed_incremental.csv'
db_dir = '/content/loaded_data'
os.makedirs(db_dir, exist_ok=True)

full_db_path = os.path.join(db_dir, 'full_data.db')
incremental_db_path = os.path.join(db_dir, 'incremental_data.db')

# Load CSVs into DataFrames
full_df = pd.read_csv(full_data_path)
incremental_df = pd.read_csv(incremental_data_path)


In [9]:
# Connect to SQLite DB for full data
conn_full = sqlite3.connect(full_db_path)

# Save to SQLite
full_df.to_sql('full_data', conn_full, if_exists='replace', index=False)
print("Full transformed data loaded into full_data.db")

# Close connection
conn_full.close()


✅ Full transformed data loaded into full_data.db


In [10]:
# Connect to SQLite DB for incremental data
conn_inc = sqlite3.connect(incremental_db_path)

# Save to SQLite
incremental_df.to_sql('incremental_data', conn_inc, if_exists='replace', index=False)
print("Incremental transformed data loaded into incremental_data.db")

# Close connection
conn_inc.close()


✅ Incremental transformed data loaded into incremental_data.db


In [11]:
# Reconnect and preview loaded full data
conn_full = sqlite3.connect(full_db_path)
print(pd.read_sql_query("SELECT * FROM full_data LIMIT 5", conn_full))

# Close
conn_full.close()


         id  diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302  Malignant        17.99         10.38          122.80     1001.0   
1    842517  Malignant        20.57         17.77          132.90     1326.0   
2  84300903  Malignant        19.69         21.25          130.00     1203.0   
3  84348301  Malignant        11.42         20.38           77.58      386.1   
4  84358402  Malignant        20.29         14.34          135.10     1297.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

   ...  texture_worst  perimeter_worst  area_worst  smoo